# Helix Python script demo
This notebook contains instructions and step by step examples on how to use functions associated to Helix v1 and troubleshooting in case of failure. This notebook should be executed one cell at a time and check that outputs are coherent.
## Setup
### Connect to Pi's rosbridge server

In [ ]:
import roslibpy
import time
import numpy as np

# Connect to the Pi's rosbridge server over port 9090
client = roslibpy.Ros(host='192.168.86.161', port=9090) # The host is the Pi's IP address
client.run()

# Check that the connecting worked
print('Is ROS connected?', client.is_connected)

If the output is: `Is ROS connected ? True`, proceed to the next step. If not, check the host and port addresses, and make sure that all docker containers on the RPi are running correctly (check documentation [here](https://github.com/helix-robotics-ag/main?tab=readme-ov-file))

### Check available topics and services (optional)

Verify that all desired topics and services are available. Usefull for a quick sanity check

In [ ]:
# Check which topics can be seen
print('ROS Topics: ', client.get_topics())

# Check which services can be seen
print('Ros Services: ', client.get_services())

## Calibration

For now the calibration is done through different functions depending on the state of the manipulator. The workflow will be roughly as follows:
1. Loosen up all tendons and make the arm roughly straight
2. Add tension to the tendons and straighten up the manipulator to it's zero state
3. Create or update existing calibration file

### Loosen up all tendons

This step is required if some tendons are tense and the arm isn't straight. Use the following service to release the tension: 

In [ ]:
# Switch to current control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_current_control', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

# Unwind tendon
srv = roslibpy.Service(client, '/tendon_transmission_node/set_unwind_current', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

Stop the motors when tendons are no longer tense:

In [ ]:
# Stop motors from turning
srv = roslibpy.Service(client, '/tendon_transmission_node/set_zero_current', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

### Add tension and straighten the manipulator

In [ ]:
# Switch to current control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_current_control', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

# Add tension
srv = roslibpy.Service(client, '/tendon_transmission_node/set_holding_current', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

### Update calibration file

After moving the manipulator to the desired zero position, save the calibration file:

In [ ]:
# Save calibration file
srv = roslibpy.Service(client, '/tendon_transmission_node/set_motor_offsets', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

# Switch to position control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_position_control', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

**Bonus**: To reuse an old calibration instead of having to straighten up the manipulator, execute this code after adding tension to the tendons:

In [ ]:
# Check old calibration
srv = roslibpy.Service(client, '/tendon_transmission_node/check_calibration', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

# Switch to position control
srv = roslibpy.Service(client, '/tendon_transmission_node/switch_to_position_control', 'std_srvs/Trigger')
set_curr_rq = roslibpy.ServiceRequest()
result = srv.call(set_curr_rq)
print(result)

## Tendon length control

Simple example using closed loop tendon length control with ROS Topics:
1. Subscribe to `/tendon_transmission_node/tendon_states` to obtain measured tendon lengths
2. Once a message is received, compute command
3. Publish commands using `/tendon_transmission_node/commands` topic. Commands are also tendon length

In [ ]:
# Subscribe to the tendon state information topic
listener = roslibpy.Topic(client, '/tendon_transmission_node/tendon_states', 'sensor_msgs/msg/JointState')
def listener_callback(message):
    print('Reading from /tendon_transmission_node/tendon_states')
    joint_names = message['name']
    tendon_positions = message['position']
    global j, jn
    # Construct a joint command in order
    jn =  sorted(joint_names) 
    j  = [tendon_positions[joint_names.index('joint0')],
          tendon_positions[joint_names.index('joint1')],
          tendon_positions[joint_names.index('joint2')],
          tendon_positions[joint_names.index('joint3')],
          tendon_positions[joint_names.index('joint4')],
          tendon_positions[joint_names.index('joint5')],
          tendon_positions[joint_names.index('joint6')]+0.02, # Increase tendon lengths (loosen) in 3rd section by 2cm
          tendon_positions[joint_names.index('joint7')]+0.02,
          tendon_positions[joint_names.index('joint8')]+0.02]
    # Stop listening after the first message
    listener.unsubscribe()
listener.subscribe(listener_callback)

# Advertise to the command topic on the tendon transmission controller
talker = roslibpy.Topic(client, '/tendon_transmission_node/commands', 'std_msgs/msg/Float64MultiArray')

# Wait to receive message on /tendon_transmission_node/tendon_states
time.sleep(1)

# The listener callback has executed by now
print('Commanding joint names: ', jn)
print('Commanding tendon positions: ', j)
print('Press any key to publish command to/tendon_transmission_node/commands')
input()
talker.publish(roslibpy.Message({'data': j}))

# Wait to make sure the command is sent before killing the client
time.sleep(1)

Another example using open loop control with slower movement (*UNTESTED*)

In [ ]:
import numpy as np
import roslibpy

# Create start position
zero = [0,0,0,0,0,0,0,0,0] 

# Create end position
j = [0,0,0,0,0,0,0,0,0]
j[0] = 0.06
j[3] = 0.06
j[6] = 0.06

# Create slowed down trajectory
T = 3 # time to move from zero to desired position (in seconds)
Update_freq = 50 # Update frequency of the commands (in Hertz) TODO: Test different update frequencies
Dxl_Positions = np.linspace(zero, j, num=T*Update_freq)

# Advertise to the command topic on the joint position controller
talker = roslibpy.Topic(client, '/tendon_transmission_node/commands', 'std_msgs/msg/Float64MultiArray')

# Publish position commands
for ele in Dxl_Positions:
    talker.publish(roslibpy.Message({'data': ele})) # Note that the controller command topic receives the joint positions in order
    time.sleep(1/Update_freq)

Terminate client after finishing the program

In [ ]:
client.terminate()